In [8]:
#function to count the syllables in a word
def count_syllables(word):

    vowels = "aeiouy"
    num_vowels = 0
    last_was_vowel = False
    for letter in word:
        if letter in vowels:
            # don't count consecutive vowels
            if not last_was_vowel:
                num_vowels += 1
            last_was_vowel = True
        else:
            last_was_vowel = False

    # handle some common exceptions
    if word[-2:] == "es" or word[-2:] == "ed":
        num_vowels -= 1
    elif word[-1:] == "e":
        num_vowels -= 1

    return max(1, num_vowels)



#inputing the required libraries

from flask import Flask , render_template , request , jsonify
from bs4 import BeautifulSoup as bs 
import pandas as pd
from urllib.request import urlopen as urReq
import requests
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os


#reading the input file
input=pd.read_excel('/Input.xlsx')

limit=input.shape[0]

#extracting the content by iterating through each link in the input file
for i in range(1,limit):
  try:
      url=input['URL'][i]
      url_id=input["URL_ID"][i]

      #getting the title of the page
      response = requests.get(url)
      soup = bs(response.content, 'html.parser')
      title = soup.title.string
      q=str(title)
      q=q.split("|")
      title=q[0]

      #getting the content of the page
      response_website = urReq(url)
      data = response_website.read()
      beautifyed_html  = bs(data,"html.parser")

      soup = bs(data, 'html.parser')

      div_tag = soup.find('div', class_='td-post-content tagdiv-type')
      if div_tag:
          p_tags = div_tag.find_all('p')
          article_text=""
          for tag in p_tags:
              article_text+=(tag.text.strip())
          final_article=title+article_text
      else:
        #some contents of the page is in a combined class
        div_tag = soup.find('div', class_='td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')
        if div_tag:
            p_tags = div_tag.find_all('p')
            article_text=""
            for tag in p_tags:
                article_text+=(tag.text.strip())
            final_article=title+article_text
        final_article
        

      #writing to the file
      with open(f"{url_id}.txt", "w") as f:
          f.write(final_article)
  except:
    #for the links which shows 404 error
    pass







#adding only those words in the extracted file if they are not found in the Stop Words Lists. 


stop_words = set()

for filename in os.listdir('/home/stopwords'):
    if filename.endswith('.txt'):
        with open(os.path.join('/home/stopwords', filename), 'r', encoding='ISO-8859-1') as f:
            stop_words.update(set(f.read().splitlines()))  # Add the stop words from the file to the set

stop_words=list(stop_words)
for i in range(len(stop_words)):
  stop_words[i]=stop_words[i].upper() #final stop word list


for filename in os.listdir('/content'):
    if filename.endswith('.txt'):
        with open(os.path.join('/content', filename), 'r') as f:
            text = f.read()
            words = word_tokenize(text)  # Tokenize the text into words
            filtered_text = [word for word in words if word.upper() not in stop_words]
            filtered_text = " ".join(filtered_text)




        with open(os.path.join('/content', filename), 'w') as f:
            f.write(filtered_text) # rewriting the file with no stop words to the same location 



positive_count = 0
negative_count = 0

positive_words = set()
with open("/home/master_dictionary/positive-words.txt", "r", encoding='ISO-8859-1') as f:
    for line in f:
        positive_words.add(line.strip())
negative_words = set()
with open("/home/master_dictionary/negative-words.txt", "r", encoding='ISO-8859-1') as f:
    for line in f:
        negative_words.add(line.strip())


for filename in os.listdir('/content'):
    if filename.endswith('.txt'):  
        with open(os.path.join('/content', filename), 'r') as f:
          text = f.read()
          index=filename.split(".")
          #finding the index of the each file name based on url_id so we edit the dataframe
          index=(int(index[0]))
          p=(input[input.URL_ID==index].index)
          index=int(p[0]) 

          #positive and negative
          words = text.split()
          for word in words:
              if word in positive_words:
                  positive_count += 1
              elif word in negative_words:
                  negative_count += 1

          input.loc[index, 'positive_score'] = positive_count   #creating a new column in the input dataframe and inputting values 
          input.loc[index, 'negative_score'] = negative_count

#polarity subjectivity
          Polarity_score = (positive_count - negative_count)/ ((positive_count + negative_count) + 0.000001)
          input.loc[index, 'Polarity score'] = Polarity_score

          total_words_after_cleaning=len(words)
          Subjectivity_Score = (positive_count+ negative_count)/ ((total_words_after_cleaning) + 0.000001)
          input.loc[index, 'Subjectivity score'] = Subjectivity_Score


#avg sentence length

          sentences = re.split(r'[.!?]+', text) #separating the text into sentences
          total_sentences = len(sentences)
          total_words_after_cleaning=len(words)
          Average_sentence_length=total_words_after_cleaning/total_sentences
          input.loc[index, 'Average_sentence_length'] = Average_sentence_length

#PERCENTAGE OF COMPLEX WORDS
          complex_words = re.findall(r'\b\w{3,}[^aeiou\s\d]*[aeiouy][^aeiou\s]*\b', text) #looking for atlest two vowels in a word 
          complex_no=len(complex_words)
          input.loc[index, 'complex word count'] = complex_no

#fog index
          percentage_complex = (complex_no/total_words_after_cleaning)
          Fog_Index = 0.4 * (Average_sentence_length + percentage_complex)
          input.loc[index, 'Fog_index'] = Fog_Index


#avg no.of words per sentences
          sentences = re.split(r'[.!?]+', text)
          total_sentences = len(sentences)
          words_per_sentence = [len(sentence.split()) for sentence in sentences]
          avg_words_per_sentence = sum(words_per_sentence) / len(words_per_sentence)
          input.loc[index, 'avg_words_per_sentence'] = avg_words_per_sentence
    

#complex word count
          input.loc[index, 'percentage of complex words'] = percentage_complex


#WORD COUNT
          sentences=nltk.sent_tokenize(text)
          corpus=""
          for i in range(len(sentences)):
            review=re.sub('[^a-zA-Z]',' ',sentences[i]) # excluding all special characters
            review=review.lower()
            review=review.split()
            review=[word for word in review if not word in set(stopwords.words('english'))] #looking for words present in the stopwords of english
            review=' '.join(review)
            corpus+=review
          word_count=len(corpus)
          input.loc[index, 'word count'] = word_count


#SYLLABLE PER WORD
          words = text.split()
          syllables = [count_syllables(word.lower()) for word in words] #calling function to find the syllable count
          total_syllables = sum(syllables)
          input.loc[index, 'total_syllables'] = total_syllables




#PERSONAL PRONOUNS
          words = text.split()
          #finding personal pronouns in the file (which do not have the stop words from the stop words of the given Stopwords folder)
          pronoun_count = re.compile(r'\b(I|we|ours|my|mine|(?-i:us))\b', re.I)
          pronouns = pronoun_count.findall(text)
          Personal_pronouns=len(pronouns)
          input.loc[index, 'Personal_pronouns'] = Personal_pronouns

#AVG WORD LENGTH
          words = text.split()
          total_chars = sum(len(word) for word in words)
          total_words = len(words)
          average_word_length = total_chars / total_words
          input.loc[index, 'average_word_length'] = average_word_length





#creating output file in csv format

input.to_csv('output.csv', index=False)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
